In [ ]:
!pip install -q -r requirements.txt

In [1]:
import os
import re
import requests
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk



In [2]:
# Download NLTK data files
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import os
env_vars = !cat .env
for var in env_vars:
    key, value = var.split('=')
    print(key)
    #print(value)
    os.environ[key] = value

ALPHA_VANTAGE_API_KEY
DEEPSEEK_API_KEY
OPENAI_API_KEY
SERPER_API_KEY
WORDPRESS_URL
WORDPRESS_USERNAME
WORDPRESS_PASSWORD


In [4]:
# Function to read the file
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()


In [5]:
# Function to preprocess text: remove punctuation, lowercase, remove stopwords
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(words)


In [6]:
# Function to extract keywords
def extract_keywords(text, num_keywords=10):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
    word_counts = Counter(filtered_words)
    return [word for word, _ in word_counts.most_common(num_keywords)]

In [7]:
# Function to generate long-tail keywords using SERPER API
def serper_api_keywords(keyword):
    api_key = "YOUR_SERPER_API_KEY"
    url = "https://serpapi.com/search"
    params = {
        "q": keyword,
        "api_key": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    long_tail_keywords = []
    if "related_searches" in data:
        for item in data["related_searches"]:
            long_tail_keywords.append(item["query"])
    return long_tail_keywords


In [8]:
# Function to generate long-tail keywords based on context
def generate_long_tail_keywords(keywords, context):
    long_tail_keywords = []
    for keyword in keywords:
        for ctx in context:
            long_tail_keywords.append(f"{keyword} {ctx}")
    return long_tail_keywords

In [9]:
# Function to generate SEO content
def generate_seo_content(text, context):
    summary = text[:5000] + "..."  # Take the first 1000 characters as a summary
    cleaned_summary_text = preprocess_text(summary)
    
    # Extract keywords from the text
    keywords = extract_keywords(cleaned_summary_text)
    
    # Generate long-tail keywords using SERPER API
    additional_keywords = []
    for keyword in keywords:
        additional_keywords.extend(serper_api_keywords(keyword))
    
    # Generate long-tail keywords based on context
    long_tail_keywords = generate_long_tail_keywords(keywords, context)
    
    all_keywords = list(set(keywords + additional_keywords + long_tail_keywords))[:30]  # Ensure unique and take only top 30
    
    seo_content = []
    #seo_content.append("Summary:\n")
    #seo_content.append(summary + "\n\n")
    seo_content.append("SEO Keywords:\n")
    seo_content.append(', '.join(all_keywords) + "\n")
    
    return ''.join(seo_content)

In [10]:
# Function to write the SEO content to a file
def write_seo_file(seo_content, output_file):
    with open(output_file, 'w') as file:
        file.writelines(seo_content)

# Main function to execute the tasks
def main():
    # Step 1: Read the report file
    report = read_file('report.txt')
    context = [
    "today's forex sentiment",
    "morning forex trading",
    "afternoon forex analysis",
    "evening forex trends",
    "forex market updates",
    "daily forex predictions",
    "forex trading tips",
    "forex risk management",
    "forex technical analysis",
    "forex fundamental analysis"
]

    
    # Step 2: Generate SEO content
    seo_content = generate_seo_content(report, context)
    
    # Step 3: Write the SEO content to a new file
    write_seo_file(seo_content, 'seo_report.md')

if __name__ == "__main__":
    main()